In [2]:
import os
import piexif
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
from scipy.stats import entropy
import cv2

In [ ]:
valid_exts = (".jpg", ".jpeg", ".png", ".tif", ".tiff")

def extract_exif_info(image_path, img_format):
   if img_format not in ["JPEG", "TIFF"]:
       return {"Make": None, "Model": None, "Software": None, "DateTimeOriginal": None}
   try:
       exif_dict = piexif.load(image_path)
       exif_data = {}
       for ifd in exif_dict:
           for tag in exif_dict[ifd]:
               key = piexif.TAGS[ifd][tag]["name"]
               value = exif_dict[ifd][tag]
               if isinstance(value, bytes):
                   value = value.decode(errors="ignore")
               exif_data[key] = value
       return {
           "Make": exif_data.get("Make", None),
           "Model": exif_data.get("Model", None),
           "Software": exif_data.get("Software", None),
           "DateTimeOriginal": exif_data.get("DateTimeOriginal", None)
       }
   except:
       return {"Make": None, "Model": None, "Software": None, "DateTimeOriginal": None}

def extract_qtable(image_path, img_format):
   if img_format != "JPEG":
       return {"qtable_mean": None, "qtable_std": None, "qtable_max": None, "qtable_min": None}
   try:
       img = Image.open(image_path)
       qt = img.quantization
       if not qt:
           return {"qtable_mean": None, "qtable_std": None}
       q = list(qt.values())[0]
       q = np.array(q)
       return {
           "qtable_mean": np.mean(q),
           "qtable_std": np.std(q),
           "qtable_max": np.max(q),
           "qtable_min": np.min(q)
       }
   except:
       return {"qtable_mean": None, "qtable_std": None, "qtable_max": None, "qtable_min": None}

def extract_entropy(img_array):
   r, g, b = img_array[:, :, 0], img_array[:, :, 1], img_array[:, :, 2]
   return {
       "r_entropy": entropy(np.histogram(r, bins=256)[0] + 1e-7),
       "g_entropy": entropy(np.histogram(g, bins=256)[0] + 1e-7),
       "b_entropy": entropy(np.histogram(b, bins=256)[0] + 1e-7)
   }

def extract_blur_sharpness(img_array):
   gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
   lap_var = cv2.Laplacian(gray, cv2.CV_64F).var()
   return {"blur_metric": lap_var}

def extract_hu_moments(img_array):
   gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
   moments = cv2.moments(gray)
   hu_moments = cv2.HuMoments(moments).flatten()
   return {f"hu_{i+1}": float(val) for i, val in enumerate(hu_moments)}

def extract_dct_features(img_array):
    gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
    h, w = gray.shape
    # Crop to even dimensions if necessary
    if h % 2 != 0:
        gray = gray[:-1, :]
    if w % 2 != 0:
        gray = gray[:, :-1]
    dct = cv2.dct(np.float32(gray) / 255.0)
    dct_abs = np.abs(dct)
    return {
        "dct_mean": np.mean(dct_abs),
        "dct_std": np.std(dct_abs),
        "dct_max": np.max(dct_abs),
        "dct_energy": np.sum(dct_abs ** 2)
    }

def extract_noise_features(img_array):
   gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
   blurred = cv2.GaussianBlur(gray, (3, 3), 0)
   noise = gray.astype(np.float32) - blurred.astype(np.float32)
   return {
       "noise_mean": np.mean(noise),
       "noise_std": np.std(noise)
   }

def extract_edge_density(img_array):
   gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
   edges = cv2.Canny(gray, 100, 200)
   return {
       "edge_density": np.sum(edges > 0) / edges.size
   }

def extract_image_stats(img):
   return {
       "width": img.width,
       "height": img.height
   }

def process_images(folder, label):
   data = []
   for fname in tqdm(os.listdir(folder), desc=f"Processing {folder}"):
       if not fname.lower().endswith(valid_exts):
           continue
       full_path = os.path.join(folder, fname)
       try:
           img = Image.open(full_path).convert("RGB")
           img_array = np.array(img)
           img_format = img.format or os.path.splitext(fname)[-1].replace(".", "").upper()

           row = {
               "filename": fname,
               "label": label,
               "format": img_format,
               "has_exif": img_format in ['JPEG', 'TIFF'],
               "has_qtable": img_format == 'JPEG'
           }

           row.update(extract_exif_info(full_path, img_format))
           row.update(extract_image_stats(img))
           row.update(extract_entropy(img_array))
           row.update(extract_blur_sharpness(img_array))
           row.update(extract_hu_moments(img_array))
           row.update(extract_dct_features(img_array))
           row.update(extract_noise_features(img_array))
           row.update(extract_edge_density(img_array))
           row.update(extract_qtable(full_path, img_format))

           data.append(row)
       except Exception as e:
           print(f"[ERROR] {fname}: {e}")
   return data

In [8]:
# Folder to label mapping
folder_label_map = {
    'human': 0,
    'edited': 1,
    'AI': 2
}

all_data = []

for folder, label in folder_label_map.items():
    folder_path = f'{folder}'
    data = process_images(folder_path, label)
    all_data.extend(data)

# Convert to DataFrame and save
df = pd.DataFrame(all_data)
df.to_csv('image_detection_metadata.csv', index=False)
print('Metadata extraction complete. Saved to image_detection_metadata.csv')

Processing AI: 100%|██████████| 39975/39975 [14:52<00:00, 44.81it/s]


Metadata extraction complete. Saved to image_detection_metadata.csv


In [64]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [65]:
import os

# Set working directory temporarily for this notebook session
os.chdir('/Users/wei/Downloads/NLP/image_detection')
print("Current working directory:", os.getcwd())

Current working directory: /Users/wei/Downloads/NLP/image_detection


In [66]:
df = pd.read_csv('image_detection_metadata.csv')

In [67]:
# For stage 1: real (0) vs not real (1)
df_stage1 = df.copy()
df_stage1['label_stage1'] = df_stage1['label'].apply(lambda x: 0 if x == 0 else 1)
df_stage1.to_csv('train_meta_stage1.csv', index=False)
# For stage 2: only edited (1) and AI (2)
df_stage2 = df[df['label'] != 0].copy()
df_stage2['label_stage2'] = df_stage2['label'].apply(lambda x: 0 if x == 1 else 1)
df_stage2.to_csv('train_meta_stage2.csv', index=False)

In [68]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load stage 1 data
df1 = pd.read_csv('train_meta_stage1.csv')

# Stratified split: 60% train, 20% val, 20% test
trainval_df1, test_df1 = train_test_split(df1, test_size=0.2, stratify=df1['label_stage1'], random_state=42)
train_df1, val_df1 = train_test_split(trainval_df1, test_size=0.25, stratify=trainval_df1['label_stage1'], random_state=42)


# Example for stage 1
meta_features1 = [col for col in train_df1.columns if col not in ['filename', 'label', 'format', 'label_stage1', 'folder']]

scaler1 = StandardScaler()
train_df1[meta_features1] = scaler1.fit_transform(train_df1[meta_features1])
val_df1[meta_features1] = scaler1.transform(val_df1[meta_features1])
test_df1[meta_features1] = scaler1.transform(test_df1[meta_features1])

# Save scaler1 for later use
import joblib
joblib.dump(scaler1, 'metadata_scaler_stage1.pkl')
# Save splits
train_df1.to_csv('train_meta_stage1_train.csv', index=False)
val_df1.to_csv('train_meta_stage1_val.csv', index=False)
test_df1.to_csv('train_meta_stage1_test.csv', index=False)

# Load stage 2 data
df2 = pd.read_csv('train_meta_stage2.csv')

# Stratified split: 60% train, 20% val, 20% test
trainval_df2, test_df2 = train_test_split(df2, test_size=0.2, stratify=df2['label_stage2'], random_state=42)
train_df2, val_df2 = train_test_split(trainval_df2, test_size=0.25, stratify=trainval_df2['label_stage2'], random_state=42)
meta_features2 = [col for col in train_df2.columns if col not in ['filename', 'label', 'format', 'label_stage2', 'folder']]

scaler2 = StandardScaler()
train_df2[meta_features2] = scaler2.fit_transform(train_df2[meta_features2])
val_df2[meta_features2] = scaler2.transform(val_df2[meta_features2])
test_df2[meta_features2] = scaler2.transform(test_df2[meta_features2])

# Save scaler1 for later use
import joblib
joblib.dump(scaler2, 'metadata_scaler_stage2.pkl')
# Save splits
train_df2.to_csv('train_meta_stage2_train.csv', index=False)
val_df2.to_csv('train_meta_stage2_val.csv', index=False)
test_df2.to_csv('train_meta_stage2_test.csv', index=False)

/Users/wei/miniforge3/envs/nlp_env/lib/python3.10/site-packages/sklearn/utils/extmath.py:1144: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/wei/miniforge3/envs/nlp_env/lib/python3.10/site-packages/sklearn/utils/extmath.py:1149: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/wei/miniforge3/envs/nlp_env/lib/python3.10/site-packages/sklearn/utils/extmath.py:1169: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
/Users/wei/miniforge3/envs/nlp_env/lib/python3.10/site-packages/sklearn/utils/extmath.py:1144: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/wei/miniforge3/envs/nlp_env/lib/python3.10/site-packages/sklearn/utils/extmath.py:1149: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/wei/mi

In [70]:
# # Load your stage 1 CSVs
# train_df1 = pd.read_csv('train_meta_stage1_train.csv')
# val_df1 = pd.read_csv('train_meta_stage1_val.csv')
# test_df1 = pd.read_csv('train_meta_stage1_test.csv')

# Add folder column based on original label
def get_folder(label):
    if label == 0:
        return 'human'
    elif label == 1:
        return 'edited'
    else:
        return 'AI'

train_df1['folder'] = train_df1['label'].apply(get_folder)
val_df1['folder'] = val_df1['label'].apply(get_folder)
test_df1['folder'] = test_df1['label'].apply(get_folder)

# Save updated CSVs
train_df1.to_csv('train_meta_stage1_train.csv', index=False)
val_df1.to_csv('train_meta_stage1_val.csv', index=False)
test_df1.to_csv('train_meta_stage1_test.csv', index=False)

In [71]:
# train_df2 = pd.read_csv('train_meta_stage2_train.csv')
# val_df2 = pd.read_csv('train_meta_stage2_val.csv')
# test_df2 = pd.read_csv('train_meta_stage2_test.csv')

# Add folder column based on original label
def get_folder(label):
    if label == 0:
        return 'human'
    elif label == 1:
        return 'edited'
    else:
        return 'AI'

train_df2['folder'] = train_df2['label'].apply(get_folder)
val_df2['folder'] = val_df2['label'].apply(get_folder)
test_df2['folder'] = test_df2['label'].apply(get_folder)

# Save updated CSVs
train_df2.to_csv('image_detection/train_meta_stage2_train.csv', index=False)
val_df2.to_csv('image_detection/train_meta_stage2_val.csv', index=False)
test_df2.to_csv('image_detection/train_meta_stage2_test.csv', index=False)

In [73]:
%cd /Users/wei/Downloads/NLP
from image_detection.image_metadata_utils import extract_metadata_from_pil_image, HybridNet, transform, HybridImageDataset

/Users/wei/Downloads/NLP


In [75]:
from torch.utils.data import WeightedRandomSampler

labels = train_df2['label_stage2'].astype(int).values
class_sample_counts = np.bincount(labels)  # [num_class_0, num_class_1]
weights = 1. / class_sample_counts         # inverse frequency
sample_weights = weights[labels]

# Use weighted sampler to balance training
sampler = WeightedRandomSampler(sample_weights,
                                num_samples=len(sample_weights),
                                replacement=True)

In [ ]:
from torch.utils.data import DataLoader
%cd /Users/wei/Downloads/NLP

# --- Stage 1: real vs not real ---
meta_features1 = [col for col in pd.read_csv('image_detection/train_meta_stage1_train.csv').columns if col not in ['filename', 'label', 'format', 'label_stage1', 'folder']]
train_dataset1 = HybridImageDataset('image_detection', 'image_detection/train_meta_stage1_train.csv', transform, meta_features1, label_col='label_stage1')
val_dataset1   = HybridImageDataset('image_detection', 'image_detection/train_meta_stage1_val.csv', transform, meta_features1, label_col='label_stage1')
train_loader1 = DataLoader(train_dataset1, batch_size=32, shuffle=True, num_workers=0)
val_loader1   = DataLoader(val_dataset1, batch_size=32, shuffle=False, num_workers=0)

# --- Stage 2: edited vs AI ---
meta_features2 = [col for col in pd.read_csv('image_detection/train_meta_stage2_train.csv').columns if col not in ['filename', 'label', 'format', 'label_stage2', 'folder']]
train_dataset2 = HybridImageDataset('image_detection', 'image_detection/train_meta_stage2_train.csv', transform, meta_features2, label_col='label_stage2')
val_dataset2   = HybridImageDataset('image_detection', 'image_detection/train_meta_stage2_val.csv', transform, meta_features2, label_col='label_stage2')
train_loader2 = DataLoader(train_dataset2, batch_size=32, sampler=sampler, num_workers=0)
val_loader2   = DataLoader(val_dataset2, batch_size=32, shuffle=False, num_workers=0)

# --- Model definitions ---
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS device")
else:
    device = torch.device("cpu")
    print("Using CPU device")
# model1 = HybridNet(num_metadata_features=len(meta_features1), num_classes=2).to(device)
model2 = HybridNet(num_metadata_features=len(meta_features2), num_classes=2).to(device)

/Users/wei/Downloads/NLP
Using MPS device


/Users/wei/miniforge3/envs/nlp_env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/wei/miniforge3/envs/nlp_env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [78]:
from tqdm import tqdm 
def train_model(model, train_loader, val_loader, device, num_epochs=20, lr=1e-4, patience=5, save_path='best_model.pth', criterion=None):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    if criterion is None:
        criterion = torch.nn.CrossEntropyLoss()
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for images, metas, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
            images, metas, labels = images.to(device), metas.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images, metas)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
        train_loss /= len(train_loader.dataset)
        print(f"Epoch {epoch+1} Train Loss: {train_loss:.4f}")

        model.eval()
        val_loss = 0
        correct = 0
        with torch.no_grad():
            for images, metas, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]"):
                images, metas, labels = images.to(device), metas.to(device), labels.to(device)
                outputs = model(images, metas)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                preds = outputs.argmax(1)
                correct += (preds == labels).sum().item()
        val_loss /= len(val_loader.dataset)
        val_acc = correct / len(val_loader.dataset)
        print(f"Epoch {epoch+1} Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            best_model_state = model.state_dict()
            torch.save(best_model_state, save_path)
            print("Validation loss improved, saving model.")
        else:
            epochs_no_improve += 1
            print(f"No improvement for {epochs_no_improve} epoch(s).")
            if epochs_no_improve >= patience:
                print(f"Early stopping triggered after {epoch+1} epochs.")
                break

    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    return model


# --- Train model2 (with class weighting) ---

model2 = train_model(model2, train_loader2, val_loader2, device, num_epochs=20, lr=1e-4, patience=5, save_path='hybrid_stage2.pth'
)

Epoch 1/20 [Train]: 100%|██████████| 846/846 [07:04<00:00,  1.99it/s]


Epoch 1 Train Loss: 0.0562


Epoch 1/20 [Val]: 100%|██████████| 282/282 [01:04<00:00,  4.39it/s]


Epoch 1 Val Loss: 0.0128 | Val Acc: 0.9963
Validation loss improved, saving model.


Epoch 2/20 [Train]: 100%|██████████| 846/846 [07:03<00:00,  2.00it/s]


...

Epoch 11 Val Loss: 0.0078 | Val Acc: 0.9979
No improvement for 4 epoch(s).


Epoch 12/20 [Train]: 100%|██████████| 846/846 [1:38:57<00:00,  7.02s/it]    


Epoch 12 Train Loss: 0.0011


Epoch 12/20 [Val]: 100%|██████████| 282/282 [16:52<00:00,  3.59s/it]    

Epoch 12 Val Loss: 0.0070 | Val Acc: 0.9983
No improvement for 5 epoch(s).
Early stopping triggered after 12 epochs.


In [ ]:
from tqdm import tqdm 
# --- Training function (from previous message) ---
def train_model(model, train_loader, val_loader, device, num_epochs=20, lr=1e-4, patience=5, save_path='best_model.pth'):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None
    if criterion is None:
        criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for images, metas, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
            images, metas, labels = images.to(device), metas.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images, metas)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
        train_loss /= len(train_loader.dataset)
        print(f"Epoch {epoch+1} Train Loss: {train_loss:.4f}")

        model.eval()
        val_loss = 0
        correct = 0
        with torch.no_grad():
            for images, metas, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]"):
                images, metas, labels = images.to(device), metas.to(device), labels.to(device)
                outputs = model(images, metas)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                preds = outputs.argmax(1)
                correct += (preds == labels).sum().item()
        val_loss /= len(val_loader.dataset)
        val_acc = correct / len(val_loader.dataset)
        print(f"Epoch {epoch+1} Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            best_model_state = model.state_dict()
            torch.save(best_model_state, save_path)
            print("Validation loss improved, saving model.")
        else:
            epochs_no_improve += 1
            print(f"No improvement for {epochs_no_improve} epoch(s).")
            if epochs_no_improve >= patience:
                print(f"Early stopping triggered after {epoch+1} epochs.")
                break

    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    return model
# --- Train both models ---
model1 = train_model(model1, train_loader1, val_loader1, device, num_epochs=20, lr=1e-4, patience=5, save_path='hybrid_stage1.pth')
model2 = train_model(model2, train_loader2, val_loader2, device, num_epochs=20, lr=1e-4, patience=5, save_path='hybrid_stage2.pth')

Epoch 1/20 [Train]:   0%|          | 0/1596 [00:00<?, ?it/s]

Epoch 1/20 [Train]: 100%|██████████| 1596/1596 [27:22<00:00,  1.03s/it]   


Epoch 1 Train Loss: 0.1371


Epoch 1/20 [Val]: 100%|██████████| 532/532 [01:56<00:00,  4.58it/s]


Epoch 1 Val Loss: 0.0675 | Val Acc: 0.9762
Validation loss improved, saving model.


...

Epoch 19 Val Loss: 0.0000 | Val Acc: 0.8864
Validation loss improved, saving model.


Epoch 20/20 [Train]: 100%|██████████| 846/846 [07:09<00:00,  1.97it/s]


Epoch 20 Train Loss: 0.0000


Epoch 20/20 [Val]: 100%|██████████| 282/282 [00:57<00:00,  4.89it/s]


Epoch 20 Val Loss: 0.0000 | Val Acc: 0.8864
Validation loss improved, saving model.


In [79]:
import numpy as np
from tqdm import tqdm
from sklearn.metrics import f1_score
import json
import torch

def optimize_thresholds(model, val_loader, device, num_classes, out_json):
    model.eval()
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for images, metas, labels in tqdm(val_loader, desc="Collecting validation predictions"):
            images, metas = images.to(device), metas.to(device)
            outputs = model(images, metas)
            probs = torch.softmax(outputs, dim=1).cpu().numpy()
            all_probs.append(probs)
            all_labels.append(labels.cpu().numpy())

    all_probs = np.concatenate(all_probs, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    best_thresholds = [0.5] * num_classes
    best_f1s = [0] * num_classes

    for class_idx in range(num_classes):
        for thresh in np.arange(0.1, 0.9, 0.01):
            preds = []
            for prob in all_probs:
                if prob[class_idx] > thresh:
                    preds.append(class_idx)
                else:
                    preds.append(np.argmax(prob))
            f1 = f1_score(all_labels, preds, average='macro')
            if f1 > best_f1s[class_idx]:
                best_f1s[class_idx] = f1
                best_thresholds[class_idx] = thresh

    print(f"Best thresholds: {best_thresholds}")
    print(f"Best F1s: {best_f1s}")

    # Save thresholds
    thresholds_dict = {f"class_{i}": float(best_thresholds[i]) for i in range(num_classes)}
    with open(out_json, "w") as f:
        json.dump(thresholds_dict, f)
    print(f"Saved best thresholds to {out_json}")
    return best_thresholds

In [ ]:
# For stage 1 (3 classes: human, edited, ai)
best_thresholds1 = optimize_thresholds(model1, val_loader1, device, num_classes=2, out_json="best_thresholds_model1.json")



Best thresholds: [np.float64(0.4899999999999998), np.float64(0.38999999999999985)]
Best F1s: [0.9853159415932478, 0.9855494164270131]
Saved best thresholds to best_thresholds_model1.json


Best thresholds: [np.float64(0.1), np.float64(0.1)]
Best F1s: [0.46987951807228917, 0.46987951807228917]
Saved best thresholds to best_thresholds_model2.json


In [80]:
# For stage 2 (2 classes: edited, ai)
best_thresholds2 = optimize_thresholds(model2, val_loader2, device, num_classes=2, out_json="image_detection/best_thresholds_model2.json")

Best thresholds: [np.float64(0.3199999999999999), np.float64(0.1)]
Best F1s: [0.996154132431603, 0.9964181935849987]
Saved best thresholds to image_detection/best_thresholds_model2.json


In [81]:
def predict_with_thresholds(probs, threshold_list):
    pred = np.argmax(probs)
    for idx, thresh in enumerate(threshold_list):
        if thresh is not None and probs[idx] > thresh:
            pred = idx
            break
    return pred

In [ ]:
# Save model 1 (real vs not real)
torch.save(model1.state_dict(), "image_detection/model1_real_vs_not_real.pth")



In [82]:
# Save model 2 (edited vs ai)
torch.save(model2.state_dict(), "image_detection/model2_edited_vs_ai.pth")